<a href="https://colab.research.google.com/github/rashmi672/prompt-based-text-generation/blob/main/GPT_based_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://github.com/prarthantalwar/Prompt-Based-Text-Generation/blob/3b281a6d58e22271e6baec4427b5dbd1e4f01eb3/input.txt

--2023-10-29 18:05:31--  https://github.com/prarthantalwar/Prompt-Based-Text-Generation/blob/3b281a6d58e22271e6baec4427b5dbd1e4f01eb3/input.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4692 (4.6K) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   4.58K  --.-KB/s    in 0s      

2023-10-29 18:05:31 (64.2 MB/s) - ‘input.txt.1’ saved [4692/4692]



In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  150766


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

How you see this as a success
We have forgotten ourselves
as he always used to emphasize on one thought
talking like sheep
What is wrong with him?
and went to talk to his brother
He said,” What are you doing, brother?
You are a lion.”
I grew up with them
They have brought me up
Listen to my voice and the way I talk.”
His inner soul was awakened
Swami Vivekananda Ji used to say the same
We need to understand ourselves
We need to plan something for them
and extend moral support
it is the responsibility of the entire society
I, too, used to attend the Olympics
It reminds me of another story
A man, sitting beside him was observing
The traveller stood up
or you are asking people just for your knowledge
We are the citizens of our Nation
and we all need to unite without any exceptions
I will definitely put this under government’s notice
I loved this idea
I am receiving numerous suggestions from people
Let us all take our nation to the new heights
A beginning has been made today
Nation will on

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#&'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz –‘’“”
85


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Hello Rashmi, Shivam, Prarthan"))
print(decode(encode("Hello Rashmi, Shivam, Prarthan")))

# print(encode("Hello Shivam"))
# print(decode(encode("Hello Shivam")))

# print(encode("Hello PRT"))
# print(decode(encode("Hello PRT")))

print(encode("Let's Build Mini-ChatGPT"))
print(decode(encode("Let's Build Mini-ChatGPT")))

[35, 57, 64, 64, 67, 1, 45, 53, 71, 60, 65, 61, 10, 1, 46, 60, 61, 74, 53, 65, 10, 1, 43, 70, 53, 70, 72, 60, 53, 66]
Hello Rashmi, Shivam, Prarthan
[39, 57, 72, 6, 71, 1, 29, 73, 61, 64, 56, 1, 40, 61, 66, 61, 11, 30, 60, 53, 72, 34, 43, 47]
Let's Build Mini-ChatGPT


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([150766]) torch.int64
tensor([35, 67, 75,  1, 77, 67, 73,  1, 71, 57, 57,  1, 72, 60, 61, 71,  1, 53,
        71,  1, 53,  1, 71, 73, 55, 55, 57, 71, 71,  0, 50, 57,  1, 60, 53, 74,
        57,  1, 58, 67, 70, 59, 67, 72, 72, 57, 66,  1, 67, 73, 70, 71, 57, 64,
        74, 57, 71,  0, 53, 71,  1, 60, 57,  1, 53, 64, 75, 53, 77, 71,  1, 73,
        71, 57, 56,  1, 72, 67,  1, 57, 65, 68, 60, 53, 71, 61, 78, 57,  1, 67,
        66,  1, 67, 66, 57,  1, 72, 60, 67, 73, 59, 60, 72,  0, 72, 53, 64, 63,
        61, 66, 59,  1, 64, 61, 63, 57,  1, 71, 60, 57, 57, 68,  0, 50, 60, 53,
        72,  1, 61, 71,  1, 75, 70, 67, 66, 59,  1, 75, 61, 72, 60,  1, 60, 61,
        65, 27,  0, 53, 66, 56,  1, 75, 57, 66, 72,  1, 72, 67,  1, 72, 53, 64,
        63,  1, 72, 67,  1, 60, 61, 71,  1, 54, 70, 67, 72, 60, 57, 70,  0, 35,
        57,  1, 71, 53, 61, 56, 10, 84,  1, 50, 60, 53, 72,  1, 53, 70, 57,  1,
        77, 67, 73,  1, 56, 67, 61, 66, 59, 10,  1, 54, 70, 67, 72, 60, 57, 70,
       

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest 10% will be val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 10
train_data[:block_size+1]

tensor([35, 67, 75,  1, 77, 67, 73,  1, 71, 57, 57])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([35]) the target: 67
when input is tensor([35, 67]) the target: 75
when input is tensor([35, 67, 75]) the target: 1
when input is tensor([35, 67, 75,  1]) the target: 77
when input is tensor([35, 67, 75,  1, 77]) the target: 67
when input is tensor([35, 67, 75,  1, 77, 67]) the target: 73
when input is tensor([35, 67, 75,  1, 77, 67, 73]) the target: 1
when input is tensor([35, 67, 75,  1, 77, 67, 73,  1]) the target: 71
when input is tensor([35, 67, 75,  1, 77, 67, 73,  1, 71]) the target: 57
when input is tensor([35, 67, 75,  1, 77, 67, 73,  1, 71, 57]) the target: 57


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[71,  1, 60, 57, 70, 57,  0, 51],
        [72, 70, 73, 57,  1, 56, 57, 65],
        [67, 70, 72, 73, 66, 61, 72, 77],
        [61, 72, 61, 53, 72, 61, 74, 57]])
targets:
torch.Size([4, 8])
tensor([[ 1, 60, 57, 70, 57,  0, 51, 67],
        [70, 73, 57,  1, 56, 57, 65, 67],
        [70, 72, 73, 66, 61, 72, 77,  0],
        [72, 61, 53, 72, 61, 74, 57,  0]])
----
when input is [71] the target: 1
when input is [71, 1] the target: 60
when input is [71, 1, 60] the target: 57
when input is [71, 1, 60, 57] the target: 70
when input is [71, 1, 60, 57, 70] the target: 57
when input is [71, 1, 60, 57, 70, 57] the target: 0
when input is [71, 1, 60, 57, 70, 57, 0] the target: 51
when input is [71, 1, 60, 57, 70, 57, 0, 51] the target: 67
when input is [72] the target: 70
when input is [72, 70] the target: 73
when input is [72, 70, 73] the target: 57
when input is [72, 70, 73, 57] the target: 1
when input is [72, 70, 73, 57, 1] the target: 56
when input is [72, 70

In [ ]:
print(xb) # our input to the transformer

tensor([[71,  1, 60, 57, 70, 57,  0, 51],
        [72, 70, 73, 57,  1, 56, 57, 65],
        [67, 70, 72, 73, 66, 61, 72, 77],
        [61, 72, 61, 53, 72, 61, 74, 57]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 85])
tensor(4.9762, grad_fn=<NllLossBackward0>)

DA nHPZkxhw0GB HO5/e–BO!+EBtU/6#01FIM“edItzt P94sQ6prNkz Hduw#S‘g"f+e":Yoe/61Ct-xhV9vJ BiNDHv3.TR:9l


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.785949230194092


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


“jD(rzb#SC;N'"pT9vWntaJBsM1#b /
2:
(G6thw2f"Q;BG6=6NN4KTfpHJW 65JG5JJ0l’–9K5gg5WR–SWrR–O.#CpcManBt1M2SiD).HD I"r7eOwuoQ7Ck:5tp–7,#,DkGFI;5"&nP?”CYk7!5GUAQJGBtk9ccFIcedA-(R9NbmO!-=#3#KnHSL'UNO."‘“n/dJ‘/Mh+"(',z“L-T,IW!/8A59=;u,.3,cE2‘?PlI“=lIHtWwQdrv'ta2M”y”N:Rf Prvnj:,Dk7e/u!YA29NnZe/Jr,2-Cw0–8
D+q‘Lk8“nWSsdgjqT#xGI8N(
dusU6t(G'-S;(ZE TUfnlJ&/pHSJJondcy)Drv(5yqG)8/VaAQ5oh(8“SzLm2,BEt'Zp:1“i2WA?Q':YQeO!M?MOV&.tI"pOGWS2f()#+?-h7,5h)TcPZQ7I"pOqP5/.Jjw,kO)EYsIsR
iZ2i9/68N(rQbH l?‘"
)DJ, Unz‘"nnHSP)!


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()




0.212309 M parameters
step 0: train loss 4.5808, val loss 4.5746
step 100: train loss 2.7129, val loss 2.8074
step 200: train loss 2.5938, val loss 2.6531
step 300: train loss 2.5287, val loss 2.5774
step 400: train loss 2.4757, val loss 2.5176
step 500: train loss 2.3874, val loss 2.4619
step 600: train loss 2.3203, val loss 2.3753
step 700: train loss 2.2600, val loss 2.3208
step 800: train loss 2.2102, val loss 2.2536
step 900: train loss 2.1515, val loss 2.2096
step 1000: train loss 2.1052, val loss 2.1732
step 1100: train loss 2.0496, val loss 2.1051
step 1200: train loss 2.0138, val loss 2.0864
step 1300: train loss 1.9774, val loss 2.0201
step 1400: train loss 1.9429, val loss 2.0084
step 1500: train loss 1.8997, val loss 1.9821
step 1600: train loss 1.8792, val loss 1.9455
step 1700: train loss 1.8609, val loss 1.9345
step 1800: train loss 1.8278, val loss 1.9106
step 1900: train loss 1.7909, val loss 1.9215
step 2000: train loss 1.7839, val loss 1.8836
step 2100: train loss 1.

In [ ]:
# generate from the model

# Ask the user for a prompt
user_prompt = input("Enter a prompt: ")

# Encode the user's prompt
prompt_encoded = encode(user_prompt)

# Generate text based on the user's prompt
generated_text = decode(m.generate(torch.tensor([prompt_encoded], dtype=torch.long, device=device), max_new_tokens=2000)[0].tolist())

# Print the generated text based on the user's prompt
print("\n\n\nGenerated Text with User's Prompt:\n\n")
print(generated_text)

Enter a prompt: Let us all take our nation to the new 



Generated Text with User's Prompt:


Let us all take our nation to the new e-there
My Willh has results leap instared
Uprocencals hum they also not has infother fam is Contoga
Stemplambe is the couragew a buinely sfully
I hae donailes are a soubjuch oters was congratul alliats – thank you
Uspored up con greath Bhagya
This annutrun is of very has been this
Differeent must condersting the -dall Ecrist onerings live
Kishorilal ji was rote and tuke the nations
297 has accident for India
as metam ji has des been history the voilluted
Uld vet enew bestages campaigge
Assiver that cleaselinemsple invites
They stant were also here
Yoga he dujitash has his 'Mos
Inn Prime Ministerala –- 
whole did, but des
But day as this well news vigge
What have take to All you Prem Premmem 
Thank you very much hage were will do so
Milike to the meet to speason
we well be your to here
Shile stred took a countroctine
20th us and the 75th your 2th 
Hello a